In [ ]:
from ccb import io
import pandas as pd
from ccb.dataset_converters import inspect_tools
from ccb.io.dataset import SpectralBand
%load_ext autoreload
%autoreload 2

In [ ]:
benchmark_name = "converted"

data = []
for task in io.task_iterator(benchmark_name):
    if task.dataset_name == "bigearthnet": continue
    loss = task.eval_loss
    if isinstance(loss, type):
        loss = loss()

    print(task.dataset_name)

    try:
        dataset = task.get_dataset('train')
        n_train = len(dataset)
    except Exception as e:
        print(e)
        n_train = -1
    
    print("  n_train: ", n_train)
    n_classes = getattr(task.label_type, "n_classes", -1)

    task_dict = {
        'name': task.dataset_name,
        'img size': ' x '.join([ str(size) for size in task.patch_size]),
        'loss': str(loss),
        'label type': task.label_type.__class__.__name__,
        'n classes': int(n_classes),
        'n time steps': task.n_time_steps,
        'n train' : n_train,
    }
    data.append(task_dict)
    
    if task.dataset_name == "NeonTree":
        NeonTree = task

    task_dict.update(inspect_tools.summarize_band_info(task.bands_info))

column_order = ("name", "img size", "label type", 'n classes', 'n train', 'n time steps', "Bands count", "Sentinel2 count", "RGB res", "NIR res", "HS res", "Elevation res")
df = pd.DataFrame.from_records(data, index="name", columns=column_order)
pd.set_option('max_colwidth', 300)

df